# Interatomic Potential Repository Content Manager

This is a working Notebook providing a single location for adding and modifying content in the potentials.nist.gov database.

Notebook outline:

0. Setup: Python library imports and defining global parameters.

1. Citations: Load, add and modify citation information associated with potential models. Each potential should have at least one citation, with the first being the "primary" one for generating IDs. For unpublished potentials, use the unpublished citation option.

2. Potentials: Load, add and modify potential information.  A "potential" should be a unique parameterization with an associated citation. Note that saving to the database is done in step #4.

3. Implementations: Load, add and modify the implementations of a potential. An "implementation" is a specific version/representation of a potential in a specific format. Each potential can have zero, one or multiple implementations.

4. Save potentials: Review the new/modified potentials and save the content to the database.

5. Actions: Actions are used to list changes to the repository. If a potential is added/modified, a corresponding action should also be added.

6. Requests: A request is listed on the repository whenever someone asks for a model that we do not have.

## 0. Setup

### Library imports

In [1]:
import uuid
import datetime
import shutil
from pathlib import Path
import socket

import potentials

import numpy as np
import pandas as pd

# Jupyter display libraries
from IPython.display import display, HTML

### Global parameters

- __complete_potentials__ is a compiled list of the potentials whose addition/changes were uploaded to the database.  This is used by step #5 for reporting website changes.  This should be reset after creating an Action for meaningful sets of potentials.

- __db__ is the Database object for accessing and uploading records to the database.  remote_name accesses the saved "potentials" settings, which has sign-in info for https://potentials.nist.gov.

- __workspace__ is the workspace to assign to the uploaded records.  If None, then they are only accessible to the submitter until the workspace is changed.  If 'Global Public Workspace', anyone can see the records after uploading.

In [2]:
complete_potentials = []

In [3]:
db = potentials.Database(local=True, remote=True, remote_name='potentials')

In [4]:
#workspaces = db.remote_database.cdcs.get_workspaces()
#workspaces

#workspace = None
workspace = 'Global Public Workspace'

In [5]:
db.remote_database.cdcs.patch('/pid/rest/settings', data={'auto_set_pid': False})

<Response [200]>

In [16]:
db.remote_database.cdcs.patch('/pid/rest/settings', data={'auto_set_pid': True})

<Response [200]>

- - -

## 1. Citations

### 1.1 Build or load citation

#### Option #1: For existing citations and/or new dois

In [9]:
doi = '10.1103/PhysRevMaterials.7.043601'

citation = db.fetch_citation(doi, verbose=True)

Citation retrieved from CrossRef


#### Option #2: For new doi-less citations

In [ ]:
author = 'M. I. Belko and V. E. Gusakov and N. N. Dorozhkin'
year = 2010
ID = note = f'{year}--Belko-V-I-Gusakov-V-E-Dorozhkin-N-N'

citation = potentials.load_record('Citation',
    ENTRYTYPE = 'inproceedings',
    title = 'Potential EDIP for germanium: parameterization and molecular dynamics simulation of point defects',
    author = author,
    ID = ID,
    note = note,
    year = year,
    booktitle = 'Proc. IV International Conference, Materials and Structures of Modern Electronics',
    month='Sep',
    address='Minsk, Belarus', 
    pages= '15--18',
    day='23--24'
)

In [5]:
author = 'M. I. Mendelev'
year = 2022
ID = note = f'{year}--Mendelev-M-I--Ni-Al'
#title = 'Efficient "universal" shifted Lennard-Jones model for all KIM API supported species'
title = 'to be published'

citation = potentials.load_record('Citation',
    ENTRYTYPE = 'unpublished',
    title = title,
    author = author,
    ID = ID,
    note = note,
    year = year,
)
citation.build_model()

DataModelDict([('citation',
                DataModelDict([('document-type', 'unspecified'),
                               ('title', 'to be published'),
                               ('author',
                                [DataModelDict([('given-name', 'M.I.'),
                                                ('surname', 'Mendelev')])]),
                               ('publication-date',
                                DataModelDict([('year', '2022')])),
                               ('bibtex',
                                '@unpublished{2022--Mendelev-M-I--Ni-Al,\n author = {M. I. Mendelev},\n note = {2022--Mendelev-M-I--Ni-Al},\n title = {to be published},\n year = {2022}\n}\n\n')]))])

#### Option #3: From local directory

In [ ]:
libdir = Path('E:/website/LAMMPS potentials/2nn/biblib')

citations = []
for bibfile in libdir.glob('*.bib'):
    with open(bibfile) as f:
        citations.append(potentials.load_record('Citation', model=f.read()))

In [67]:
bibfile = Path('C:/Users/lmh1/Desktop/S0022459622002201-alt.bib')
with open(bibfile, encoding='UTF-8') as f:
    citation = potentials.load_record('Citation', model=f.read())

In [ ]:
citation = citations[19]
print(citation.year_authors)

#### Option #4: From currently loaded potentials

In [6]:
citation = complete_potentials[0].citations[0]
print(citation.year_authors)

IndexError: list index out of range

### 1.2 Investigate and modify

- Add abstract

- Check and replace garbled latex or other symbols in authors, title and abstract

- Verify correct year and page number(s)

#### 1.2.1 Generate html and list content fields

In [6]:
print(citation.year_authors)
print()
citation.html(render=True)
print()
for key, value in citation.bib.items():
    print(key+':', value)

2023--Nitol-M-S-Dang-K-Fensin-S-J-et-al




journal: Physical Review Materials
title: Hybrid interatomic potential for Sn
author: Mashroor S. Nitol and Khanh Dang and Saryu J. Fensin and Michael I. Baskes and Doyl E. Dickel and Christopher D. Barrett
number: 4
volume: 7
publisher: American Physical Society (APS)
month: apr
year: 2023
url: https://doi.org/10.1103%2Fphysrevmaterials.7.043601
doi: 10.1103/physrevmaterials.7.043601
ENTRYTYPE: article
ID: Nitol_2023


#### 1.2.2 Modify and update fields

Any valid bibtex fields can be added, but citation rendering relies only on the primary fields. 

In [19]:
citation.bib['author'] = 'G. Plummer and H. Rathod and A. Srivastava and M. Radovic and T. Ouisse and M. Yildizhan and P.O.Å. Persson and K. Lambrinou and M.W. Barsoum and G.J. Tucker'

In [86]:
citation.bib['title'] = 'Effect of hydrogen on plasticity of α-Fe: A multi-scale assessment'
#citation.bib['issue'] = '1'
#citation.bib['year'] = '2021' 

In [7]:
citation.bib['abstract'] = 'Interatomic potentials for the Al-Ti, Al-Ta, Al-Zr, Al-Nb and Al-Hf binary systems have been developed based on the second nearest-neighbor modified embedded-atom method (2NN MEAM) formalism. The fundamental materials properties (structural, thermodynamic and elastic behaviors of different intermetallics) could be readily described with the potentials using molecular dynamic simulation (MD), in rational agreements with experimental or first principles data. The potentials are further utilized to develop an interatomic potential for the (TiZrNbHfTa)Al<sub>3</sub> high entropy intermetallic compound (HEIC), which open the door to understand atomic scale behavior of HEICs.'

In [8]:
citation.bib['abstract'] = ' '.join([
    "To design materials for extreme applications, it is important to understand and predict phase transitions and their",
    "influence on material properties under high pressures and temperatures. Atomistic modeling can be a useful tool to",
    "assess these behaviors. However, this can be difficult due to the lack of fidelity of the interatomic potentials in",
    "reproducing this high pressure and temperature extreme behavior. Here, a hybrid EAM-R—which is the combination of",
    "embedded atom method (EAM) and rapid artificial neural network potential—for Tin (Sn) is described which is capable",
    "of accurately modeling the complex sequence of phase transitions between different metallic polymorphs as a function",
    "of pressure. This hybrid approach ensures that a basic empirical potential like EAM is used as a lower energy",
    "bound. By using the final activation function, the neural network contribution to energy must be positive, assuring",
    "stability over the whole configuration space. This implementation has the capacity to reproduce density functional",
    "theory results at 6 orders of magnitude slower than a pair potential for molecular dynamics simulation, including",
    "elastic and plastic characteristics and relative energies of each phase. Using calculations of the Gibbs free energy, it",
    "is demonstrated that the potential precisely predicts the experimentally observed phase changes at temperatures and",
    "pressures across the whole phase diagram. At 10.2 GPa, the present potential predicts a first-order phase transition",
    "between body-centered tetragonal (BCT) β-Sn and another polymorph of BCT-Sn. This structure transforms into",
    "body-centered cubic near the experimentally reported value at 33 GPa. Thus, the Sn potential developed in this paper",
    "can be used to study complex deformation mechanisms under extreme conditions of high pressure and strain rates unlike",
    "existing potentials. Moreover, the framework developed in this paper can be extended for different material systems",
    "with complex phase diagrams."])
#citation.bib['author'] = 'Ryan S. Elliott and Andrew Akerson'
#citation.bib['volume']
#citation.bib['number']
#citation.bib['title'] = 'Evaluating the performance of ReaxFF potentials for sp<sup>2</sup> carbon systems (graphene, carbon nanotubes, fullerenes) and a new ReaxFF potential'
#citation.bib['journal'] = 'Journal of Physics: Condensed Matter'
#citation.bib['year']
#citation.bib['pages'] = '951261'


In [22]:
citation.bib['doi'] = '10.1016/j.jssc.2022.123096'

In [19]:
citation.bib['pages'] = '043601'

#### 1.2.3 Review and Save Citation

In [9]:
citation.name

'10.1103_physrevmaterials.6.013608'

In [20]:
citation.build_model()
citation.html(render=True)

In [18]:
citation.model.json()

'{"citation": {"document-type": "journal", "title": "Machine learned interatomic potential for dispersion strengthened plasma facing components", "author": [{"given-name": "E.L.", "surname": "Sikorski"}, {"given-name": "M.A.", "surname": "Cusentino"}, {"given-name": "M.J.", "surname": "McCarthy"}, {"given-name": "J.", "surname": "Tranchida"}, {"given-name": "M.A.", "surname": "Wood"}, {"given-name": "A.P.", "surname": "Thompson"}], "publication-name": "The Journal of Chemical Physics", "publication-date": {"year": 2023}, "volume": 158, "issue": 11, "abstract": "Tungsten (W) is a material of choice for the divertor material due to its high melting temperature, thermal conductivity, and sputtering threshold. However, W has a very high brittle-to-ductile transition temperature, and at fusion reactor temperatures (\\u22651000 K), it may undergo recrystallization and grain growth. Dispersion-strengthening W with zirconium carbide (ZrC) can improve ductility and limit grain growth, but much 

In [21]:
db.save_citation(citation, overwrite=True, verbose=True)
db.upload_citation(citation, workspace=workspace, overwrite=True, verbose=True)

Citation record named 10.1103_physrevmaterials.7.043601 updated in C:\Users\lmh1\Documents\library
Citation record named 10.1103_physrevmaterials.7.043601 added to https://potentials.nist.gov/
Citation record named 10.1103_physrevmaterials.7.043601 assigned to workspace Global Public Workspace


In [72]:
#db.delete_citation(citation, local=True, remote=True, verbose=True)

Citation record named 10.1007_s10853-022-07118-7 deleted from C:\Users\lmh1\Documents\library
Citation record named 10.1007_s10853-022-07118-7 deleted from https://potentials.nist.gov/


In [ ]:
#db.download_citations(overwrite=True, verbose=True)

In [22]:
citations = []

In [23]:
citations.append(citation)

In [20]:
citations

- - -

## 2. Potentials

### 2.1 Build or load potential

#### Option #1: Build new potential using citation(s)

- __elements__ (*list*) The element models used by the potential.
- __citations__ (*list*) The citation(s) to associate with the potential.
- __notes__ (*str, optional*) Notes providing more information on the potential, such as usage notes and how the potential relates to other potentials.
- __key__ (*str, optional*) UUID4 key to assign to the potential.  If None, a new key will be generated.
- __othername__ (*str, optional*) Specifies an alternate description of the interactions besides the list of elements.  Examples include if the model is "universal", designed for a specific compound, or is a coarse-grain representation.
- __fictional__ (*bool, optional*) Flag indicating if the model is "fictional", i.e. purposefully fit to unrealistic properties. 
- __modelname__ (*str, optional*) Used if citation + element info is not enough to create a unique ID, e.g. a publication lists multiple parameterizations of the same element. The value should make it clear which version it is associated with, and correspond to how the authors refer to it when possible.
- __recorddate__ (*datetime.date, optional*) Date to assign to the record.  If not given, will use today's date.

In [24]:
potential = potentials.load_record('Potential',
    elements=['Sn'],
    citations=citations,
    #notes=' '.join([
    #    'Richard Jana notes that "This potential is suitable for a wide range of applications and structures, so we consider it general purpose. However, it is particularly intended for Fe nano particles."'
    #]),
    #key=None,
    #othername='Ge60Se40',
    #fictional=True,
    #modelname='1',
    #recorddate=datetime.date(2019, 12, 24)
    )
potential.build_model()
None

#### Option #2: Select existing potential

In [8]:
potential = db.get_potential(id='2023--Nitol-M-S-Dang-K-Fensin-S-J-et-al--Sn', verbose=True)

Matching record retrieved from local


F:\Python-packages\yabadaba\yabadaba\database\LocalDatabase.py:138: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cache = cache.append(newrecords, sort=False).sort_values('name')


#### Option #3: Get currently loaded potential

In [104]:
potential = complete_potentials[1]
print(potential.name)

potential.2021--Sivaraman-G-Guo-J-Ward-L-et-al--Li-Cl


### 2.2 Investigate and modify

#### 2.2.1 Generate html and list content fields

In [10]:
potential.url = f'https://potentials.nist.gov/pid/rest/local/potentials/{potential.name}'
potential.html(render=True)

In [7]:
potential.metadata()

{'name': 'potential.2014--Lee-B-M-Lee-B-J--Zr-H',
 'key': 'ebea64e3-886d-4005-9966-0d7e2bad13cc',
 'id': '2014--Lee-B-M-Lee-B-J--Zr-H',
 'url': 'https://potentials.nist.gov/pid/rest/local/potentials/potential.2014--Lee-B-M-Lee-B-J--Zr-H',
 'recorddate': datetime.date(2019, 5, 6),
 'notes': None,
 'fictional': False,
 'elements': ['Zr', 'H'],
 'othername': None,
 'modelname': None,
 'citations': [{'name': '10.1007_s11661-014-2230-4',
   'year_authors': '2014--Lee-B-M-Lee-B-J',
   'year': '2014',
   'volume': '45',
   'url': 'https://doi.org/10.1007%2Fs11661-014-2230-4',
   'title': 'A Comparative Study on Hydrogen Diffusion in Amorphous and Crystalline Metals Using a Molecular Dynamics Simulation',
   'publisher': 'Springer Nature',
   'pages': '2906--2915',
   'number': '6',
   'month': 'feb',
   'journal': 'Metallurgical and Materials Transactions A',
   'doi': '10.1007/s11661-014-2230-4',
   'author': 'Byeong-Moon Lee and Byeong-Joo Lee',
   'abstract': 'A comparative study on hydrog

#### 2.2.2 Modify and update fields

Same fields as above

In [ ]:
#potential.key = 
#potential.recorddate = datetime.date()
#potential.elements = []
#potential.othername = 'water'
#potential.fictional = 
#potential.modelname = 

In [26]:
potential.notes = "This is part of a family of potentials designed to investigate liquid and amorphous solid structures of specific compositions. This particular potential was designed for Ge<sub>0.4</sub>Se<sub>0.6</sub>."

In [9]:
potential.notes = "Due to the short-range nature of interactions, this potential should not be used for the vibrational properties of the InP description. The proposed potential also fails in modeling indium phosphide's melting temperature because this property was not included in reference data for the fitting procedure." 

In [23]:
potential.notes = ' '.join([''])

In [34]:
potential.notes = 'Update Jan 14 2022: Citation information added and id updated from 2021--Mendelev-M--Al-Sm.'

In [11]:
potential.notes

'This is a modified version of <a href="https://www.ctcms.nist.gov/potentials/entry/2009--Bonny-G-Pasianot-R-C-Castin-N-Malerba-L--Fe-Cu-Ni/">2009--Bonny-G-Pasianot-R-C-Castin-N-Malerba-L--Fe-Cu-Ni</a> that adds the ZBL correction at short distances making it suitable for collision cascade simulations. The change does not impact any of the previously published results.'

In [12]:
potential.notes = 'The current interatomic potentials are a modified version of <a href="https://www.ctcms.nist.gov/potentials/entry/2009--Bonny-G-Pasianot-R-C-Castin-N-Malerba-L--Fe-Cu-Ni/">2009--Bonny-G-Pasianot-R-C-Castin-N-Malerba-L--Fe-Cu-Ni</a>, that include the ZBL correction at short distances, making them suitable for collision cascade simulations. Also, the Ni embedding function is currently modified for densities beyond 1.5 times the equilibrium value, in order to obtain a smooth equation of state behavior. The changes do not impact any of the previously published results.'

In [36]:
potential.citations[0] = citation
potential.name = f'potential.{potential.id}'
potential.build_model()

DataModelDict([('interatomic-potential',
                DataModelDict([('key', 'f33fbb2c-82d8-45c1-8199-eac338de6861'),
                               ('id', '2021--Song-H-Mendelev-M-I--Al-Sm'),
                               ('record-version', '2021-03-15'),
                               ('description',
                                DataModelDict([('citation',
                                                DataModelDict([('document-type',
                                                                'journal'),
                                                               ('title',
                                                                'Molecular Dynamics Study of Mechanism of Solid-Liquid Interface Migration and Defect Formation in Al<sub>3</sub>Sm Alloy'),
                                                               ('author',
                                                                [DataModelDict([('given-name',
                                            

## 3. Implementations

### 3.1 List all current implementation types

List is given to help keep type values (relatively) uniform. Only needs to be called after new types have been added.

In [16]:
pots = db.get_potentials(remote=False)
imptypes = []
for pot in pots:
    for imp in pot.implementations:
        imptypes.append(imp.type)
imptypes = np.unique(imptypes)

print('All existing implementation types:')
for imptypes in imptypes:
    print('   ', imptypes)

F:\Python-packages\yabadaba\yabadaba\database\LocalDatabase.py:138: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cache = cache.append(newrecords, sort=False).sort_values('name')


All existing implementation types:
    ADP tabulated functions
    Dynamo MEAM
    EAM setfl
    EAM tabulated functions
    Equations
    FORTRAN
    Finnis-Sinclair tables
    GAP
    GULP
    IMD option EAM
    LAMMPS pair_style adp
    LAMMPS pair_style aenet (custom)
    LAMMPS pair_style agni
    LAMMPS pair_style bop
    LAMMPS pair_style comb3
    LAMMPS pair_style eam
    LAMMPS pair_style eam/alloy
    LAMMPS pair_style eam/cd
    LAMMPS pair_style eam/fs
    LAMMPS pair_style eam/he
    LAMMPS pair_style edip
    LAMMPS pair_style edip/multi
    LAMMPS pair_style eim
    LAMMPS pair_style extep
    LAMMPS pair_style hybrid table linear 1000 eam/alloy
    LAMMPS pair_style hybrid/overlay eam/alloy eam/fs
    LAMMPS pair_style hybrid/overlay zbl eam/alloy
    LAMMPS pair_style hybrid/overlay zbl snap
    LAMMPS pair_style lcbop
    LAMMPS pair_style meam
    LAMMPS pair_style meam (modified)
    LAMMPS pair_style meam/spline
    LAMMPS pair_style pinn
    LAMMPS pair_style pol

### 3.2 Build or load implementation

#### 3.2.1 List ids for current potential and implementations

In [19]:
print('suggested imp prefix:', potential.impid_prefix)
print('implementation ids:')
for i, implementation in enumerate(potential.implementations):
    print(f'{i}: {implementation.id} {implementation.type}')

suggested imp prefix: 2023--Jana-R--Fe
implementation ids:


In [19]:
#del potential.implementations[0]

#### 3.2.2 Build new implementation

- __type__ (*str*) Describes the type of the implementation. Use one of the listed values above if possible.
- __id__ (*str*) Unique human-readable id for the implementation. For new content, should be derived from the associated potential id:
    - Remove all authors except for the first
    - Add a simple format descriptor: LAMMPS, GULP, table, parameters, FORTRAN, etc.
    - Add a version descriptor: ipr1, ipr2, etc
- __notes__ (*str, optional*) Notes on the implementation.  This includes where the file(s) came from, who created them, testing info, how it differs from other versions, etc.
- __key__ (*str, optional*) UUID4 key to assign to the implementation.  If None, a new key will be generated.
- __status__ (*str, optional*) Indicates the status of the implementation. Available values are 
    - "active" (default) indicates a current implementation.
    - "superseded" indicates an implementation that is still consistent with the potential model, but has minor issues that were fixed by a newer implementation.
    - "retracted" indicates an implementation that was identified as being an invalid representation of the potential model.
- __date__ (*datetime.date, optional*) The date that the implementation was submitted or added.  If not given, will use today's date.

In [25]:
potential.add_implementation(
    type = 'LAMMPS pair_style rann (modified)',
    id = f'{potential.impid_prefix}--LAMMPS--ipr1',
    notes = ' '.join([
        'Information for this potential was provided by Mashroor Nitol on July 13, 2023.',
        'This RANN-EAM hybrid potential requires building LAMMPS with the modified rann pair_style',
        'code found at the github link below.  The link also provides instructions on LAMMPS version',
        'dependencies, build instructions, and a usage example.'
    ]),
    #key='0cdc699f-e78a-49db-8606-7a33c98c2184',
    #status=None,
    #date=datetime.date(2020, 6, 18), 
)
index = len(potential.implementations) - 1

In [28]:
kimid = 'MO_959249795837'
implementation = potentials.Implementation(
    type='OpenKIM',
    id=kimid,
    notes=' '.join(['Listing found at https://openkim.org.']),
    #notes=' '.join(['Listing found at https://openkim.org.  This KIM potential is based on a parameter file with identical parameter values as 1989--Tersoff-J--Si-Ge--LAMMPS--ipr1.']),
    #notes=' '.join(['Listing found at https://openkim.org. This KIM potential is implemented from the analytical expressions rather than a tabulated parameter file.'])#  The parameter file that this KIM potential is based on has slightly different values due to precision rounding than 2017--Purja-Pun-G-P--Si--LAMMPS--ipr1.']),
    key='da76eb36-7ca1-421a-bd64-8f6bdd905b15',
    #status=None,
    #date=datetime.date(2020, 6, 18), 
)
implementation.add_link(url=f"https://openkim.org/id/{kimid}", linktext = kimid)

index = len(potential.implementations)
potential.implementations.append(implementation)



In [26]:
potential.build_model()
potential.html(render=True)

#### 3.2.3 Select implementation

Must be done

In [27]:
index = 0
implementation = potential.implementations[index]

### 3.3 Modify implementation

#### 3.3.1 Modify implementation metadata 

In [10]:
implementation.notes

'This file was provided by Roberto Pasianot on 8 June 2023.'

In [19]:
implementation.notes = ' '.join(['These files were provided by Jyri Kimari on 8 May 2023.  The code_and_tests.zip folder contains',
                      'the fortran program and input file used to generate the eam.fs file, plots of the potential functions,'
                      'and plots of the binary alloying energies.  For the alloying energies, two sizes were investigated (256 atoms and 32000 atoms)',
                      'which respectively agree with the local and hydrostatic configurational sampling models (LCSM and HCSM)',
                      'reported in the paper.'])

In [33]:
implementation.notes = 'The current interatomic potentials are a modified version of 2009--Bonny-G-Pasianot-R-C-Castin-N-Malerba-L--Fe-Cu-Ni (available in this repository), that include the ZBL correction at short distances, making them suitable for collision cascade simulations. Also, the Ni embedding function is currently modified for densities beyond 1.5 times the equilibrium value, in order to obtain a smooth equation of state behavior. The changes do not impact any of the previously published results.'

In [9]:
implementation.notes = 'These files were provided by Zacharias Fthenakis on Nov 3, 2022. "in.graphene" and "data.graphene_H_C_O_N" provide an example LAMMPS script and corresponding atomic configuration.'

In [73]:
implementation.notes = 'These files were provided by Ganesh Sivaraman on August 6, 2022. The files are alternatively hosted at <a href="https://figshare.com/articles/dataset/Multiphase_GAP_Model_and_Training_Dataset_for_Hafnium_dioxide/14601579">https://figshare.com/articles/dataset/Multiphase_GAP_Model_and_Training_Dataset_for_Hafnium_dioxide/14601579</a>'

In [30]:
implementation.type = 'LAMMPS pair_style meam'

In [45]:
implementation.id = f'{potential.impid_prefix}--LAMMPS--ipr1'
implementation.id

'2021--Song-H--Al-Sm--LAMMPS--ipr1'

In [ ]:
#display(HTML(implementation.html()))

In [12]:
implementation.notes += " Update July 19 2023: This version has been superseded as it had the incorrect atomic mass for H."

In [11]:
implementation.status='superseded'

In [ ]:
#implementation.type='LAMMPS pair_style sw',
#implementation.key=None,
#implementation.id='2009--Molinero-V--water--ipr-1',
#implementation.status=None,
#implementation.date=datetime.date(2020, 1, 10), 
implementation.notes= ' '.join(['The table files were sent by Xianbao Duan (Huazhong Univ. of Sci. and Tech) on 18 June 2020 and posted with his permission. The example.lammps.in file gives an example of the LAMMPS pair_style and pair_coeff lines that can be used. A copy of Ta_Zhou04.eam.alloy from 2004--Zhou-X-W--Ta--LAMMPS--ipr2 is included here for completeness.',
                                ])

In [36]:
implementation.type 

'LAMMPS pair_style reaxff'

In [ ]:
implementation.date=datetime.date(2014, 11, 24)

In [27]:
implementation.notes

'These files were provided by Linus Erhard on Nov 1, 2022, and are alternatively available from <a href="https://doi.org/10.5281/zenodo.6353684">https://doi.org/10.5281/zenodo.6353684</a> (under potential) or from <a href="https://hessenbox.tu-darmstadt.de/getlink/fiCU6MLvWiE4vGEouHvLi311/">https://hessenbox.tu-darmstadt.de/getlink/fiCU6MLvWiE4vGEouHvLi311/</a>. For running the potential the <a href="https://libatoms.github.io/QUIP/">QUIP package</a> within LAMMPS is necessary. The file pot.in gives an example of the LAMMPS inputs to use to run this potential. Alternatively, the potential can be used in a python-ase interface called quippy.'

In [34]:
implementation.notes = ' '.join(["These files were provided by Linus Erhard on Nov 1, 2022, and are alternatively available",
                      'at the links listed below.',
                      'For running the potential the <a href="https://libatoms.github.io/QUIP/">QUIP package</a> within LAMMPS is necessary.',
                      'The file pot.in gives an example of the LAMMPS inputs to use to run this potential.',
                      'Alternatively, the potential can be used in a python-ase interface called quippy.'])

In [ ]:
implementation.notes += ' Update Jan 15, 2020: It was noticed that the original file hosted here was truncated and incomplete.'
implementation.notes += ' The incomplete file will not work with LAMMPS versions after 7 Aug 2019.'
implementation.notes += ' For earlier LAMMPS versions, both versions of the parameter file appear to behave identically.'

In [ ]:
implementation.status = 'retracted'
implementation.notes = 'This file was sent by J. Kullgren (Uppsala University) on 19 December 2016 and posted with his permission. Update March 15, 2020: This version was identified to not be compatible with LAMMPS.'

In [ ]:
implementation.notes='These files were posted on March 15, 2020 by Lucas Hale. They modify the above version by separating the comments into a separate file, making the parameter file compatible with LAMMPS.'

In [ ]:
implementation.type = 'LAMMPS pair_style reax/c'

In [ ]:
implementation.id = '2013--Bonny-G--Fe-Cr-W--LAMMPS--ipr1'

In [36]:
implementation.artifacts

[]

In [10]:
implementation.links.pop(1)

#### 3.3.2 Add implementation content

The content for the implementation can exist in one of the following formats

1. Artifacts: files that are hosted on the potentials repository.
2. Parameters: list of the potential's parameters to show as html text on the repository.
3. WebLinks: hyperlinks to content hosted externally.

#### Option #1: Add artifacts

File path parameters

- __newpotpath__ is the path to the directory where the new potential files are located.
- __localpath__ is the path to the local copy of the NIST potentials repository.
- __webpath__ is the url path for the NIST potentials repository that coincides with the localpath on the machine.
- __relpath__ is the relative path from localpath where the files are copied to, and the relative path from webpath where they will be found once uploaded to the website.

In [28]:
# Paths to files on desktop
machine = socket.gethostname()
if machine == 'PG903193':
    newpotpath = Path('F:/website/new potentials')
    localpath = Path('F:/website/IPR-website/potentials/')

# Paths to files on laptop
elif machine == 'PG902164':
    newpotpath = Path('C:/Users/lmh1/Documents/website/new potentials')
    localpath = Path('C:/Users/lmh1/Documents/website/IPR-website/potentials/')

# Web and relative paths
webpath = 'https://www.ctcms.nist.gov/potentials/'
relpath = f'Download/{potential.id}/{index+1}/'

Each artifact has:

- __filename__ (*str*) The name of the file (no path information).
- __url__ (*str*) The absolute url where the file can be downloaded from.
- __label__ (*str, optional*) An optional descriptive label for the file.

In [29]:
# Specify files from newpotpath to add
filenames = [
    Path(newpotpath, 'Sn.eam-r'),
    #Path(newpotpath, 'library.meam'),
    ]

labels = [None for i in range(len(filenames))]

In [24]:
# Specify files from newpotpath to add
filenames = [
    Path(newpotpath, 'iron.xml'),
    Path(newpotpath, 'iron.xml.sparseX.GAP_2022_12_5_120_12_28_45_6441'),
    Path(newpotpath, 'iron.xml.sparseX.GAP_2022_12_5_120_12_28_45_6442'),
    Path(newpotpath, 'iron.xml.sparseX.GAP_2022_12_5_120_12_28_45_6443'),
    Path(newpotpath, 'compress.dat'),
    ]

labels = [None for i in range(len(filenames))]

In [18]:
# Specify files from newpotpath to add
filenames = [
    Path(newpotpath, 'mie.mod'),
    Path(newpotpath, 'mie_Co-Co.table'),
    Path(newpotpath, 'mie_Cr-Cr.table'),
    Path(newpotpath, 'mie_Fe-Fe.table'),
    Path(newpotpath, 'mie_Mn-Mn.table'),
    Path(newpotpath, 'mie_Ni-Ni.table'),
    Path(newpotpath, 'mie_Co-Cr.table'),
    Path(newpotpath, 'mie_Co-Fe.table'),
    Path(newpotpath, 'mie_Co-Mn.table'),
    Path(newpotpath, 'mie_Co-Ni.table'),
    Path(newpotpath, 'mie_Cr-Fe.table'),
    Path(newpotpath, 'mie_Cr-Mn.table'),
    Path(newpotpath, 'mie_Cr-Ni.table'),
    Path(newpotpath, 'mie_Fe-Mn.table'),
    Path(newpotpath, 'mie_Fe-Ni.table'),
    Path(newpotpath, 'mie_Mn-Ni.table'),
    ]

labels = [
    'lammps commands',
    'Co-Co',
    'Cr-Cr',
    'Fe-Fe',
    'Mn-Mn',
    'Ni-Ni',
    'Co-Cr',
    'Co-Fe',
    'Co-Mn',
    'Co-Ni',
    'Cr-Fe',
    'Cr-Mn',
    'Cr-Ni',
    'Fe-Mn',
    'Fe-Ni',
    'Mn-Ni',
]

In [30]:
# Make download directory in local copy of website
downloadpath = Path(localpath, relpath)
if not downloadpath.is_dir():
    downloadpath.mkdir(parents=True)

# Copy files and add artifact listings
for filename, label in zip(filenames, labels):
    # Build url
    url = webpath + relpath + filename.name
    # Copy files
    shutil.copy(filename, downloadpath)
    print(Path(downloadpath, filename.name))
    
    # Add artifact listing to implementation
    implementation.add_artifact(filename=filename.name, url=url, label=label)

F:\website\IPR-website\potentials\Download\2023--Nitol-M-S-Dang-K-Fensin-S-J-et-al--Sn\1\Sn.eam-r


#### Option #1b: Update file paths

In [43]:
newdir= Path(localpath, relpath)
if not newdir.is_dir():
    newdir.mkdir(parents=True)

for artifact in implementation.artifacts:
    # Copy artifact file to new location
    oldpath = Path(localpath, artifact.url.replace(webpath, ''))
    
    newpath = Path(newdir, artifact.filename)
    #shutil.copy(oldpath, newpath)
    
    # Update the url
    artifact.url = webpath + relpath + artifact.filename

#### Option #2: Add parameters

Each parameter has:

- __name__ (*str, optional*) The name of the parameter or string parameter line.
- __value__ (*float, optional*) The value of the parameter.
- __unit__ (*str, optional*) Units associated with value.

Note: if the parameters are purely text-based, you can use only the name field.

#### Option #3: Add weblinks

Each weblink has:

- __url__ (*str*) URL for the link.
- __label__ (*str, optional*) A short description label to proceed the link.
- __linktext__ (*str, optional*) The text for the link, i.e. what gets clicked on.

In [26]:
url = 'https://doi.org/10.5281/zenodo.7630369'
label = 'zenodo, includes training data'
linktext = 'https://doi.org/10.5281/zenodo.7630369'

implementation.add_link(url=url, label=label, linktext=linktext)

In [31]:
url = 'https://github.com/MashoorNitol/Hybrid-RANN-EAM'
label = 'RANN-EAM hybrid source code and instructions'
linktext = 'https://github.com/MashoorNitol/Hybrid-RANN-EAM'

implementation.add_link(url=url, label=label, linktext=linktext)

In [82]:
implementation.links[0].linktext=linktext

## 4. Review and save potentials

In [32]:
potential.build_model()
potential.html(render=True)

In [75]:
potential.notes = 'Kun Wang notes that "This potential well reproduces the properties of Pb under finite temperatures, such as the temperature-dependence of elastic constants, linear expansion, and the high-pressure FCC-HCP phase transition at finite temperatures. The computational efficient of such potential is just slightly slower than the EAM potential."'

In [78]:
implementation.notes = 'These files were provided by Kun Wang on 8 June 2023. This potential can be used by building LAMMPS (only support LAMMPS version 16Feb16 at present) with the REAM source files (pair_ream.cpp and pair_ream.h). And two commands, i.e., "pair_style ream" and "pair_coeff * * Pb_wang.ream Pb", should be used in the LAMMPS script.'

In [33]:
db.save_potential(potential, overwrite=True, verbose=True)
db.upload_potential(potential, workspace=workspace, overwrite=True, verbose=True)
complete_potentials.append(potential)

Potential record named potential.2023--Nitol-M-S-Dang-K-Fensin-S-J-et-al--Sn added to C:\Users\lmh1\Documents\library
Potential record named potential.2023--Nitol-M-S-Dang-K-Fensin-S-J-et-al--Sn added to https://potentials.nist.gov/
Potential record named potential.2023--Nitol-M-S-Dang-K-Fensin-S-J-et-al--Sn assigned to workspace Global Public Workspace


In [28]:
complete_potentials

In [ ]:
#db.delete_potential(potential, local=True, remote=True, verbose=True)

In [ ]:
db.download_potentials(overwrite=True, format='json', indent=4, verbose=True)

In [23]:
complete_potentials.append(potential)

## 5. potential_LAMMPS

This section generates the potential_LAMMPS metadata records for generating the LAMMPS input command lines for supported potentials.

### 5.1 Build potential_LAMMPS

#### Specify parameters

Common settings

- __id__ (*str, optional*) implementation id. Should match with the implementation id from above.
- __key__ (*str, optional*) implementation key. Should match with the implementation key from above.
- __potid__ (*str, optional*) potential id. Should match with the potential id from above.
- __potkey__ (*str, optional*) potential key. Should match with the potential key from above.
- __elements__ (*list, optional*) The elements that the implementation simulates.  Should match with the elements from above if the implementation explicitly models each element.
- __symbols__ (*list, optional*) The particle model symbols defined by the implementation.  Not needed if identical to elements.
- __masses__ (*list, optional*) The masses to assign to each symbol/element.  Should be given if the implementation specifies the mass.  Required if elements is not given.
- __units__ (*str, optional*) The LAMMPS units to use with the implementation.  Default value is 'metal'.
- __atom_style__ (*str, optional*) The LAMMPS atom_style to use with the implementation.  Default value is 'atomic'.
- __pair_style__ (*str*) The LAMMPS pair_style associated with the implementation.
- __pair_style_terms__ (*list, optional*) Any additional terms that should appear on the pair_style command line.
- __comments__ (*str, optional*) Info about where the potential is hosted, to be printed when LAMMPS commands are generated.
- __dois__ (*list, optional*) A list of publication DOIs associated with the potential. 
- __artifacts__ (*list, optional*) A list of Artifact objects for the associated potential files.

In [38]:
potential.elements

['Ge', 'Se']

In [32]:
kwargs = {}
kwargs['id'] = implementation.id
kwargs['key'] = implementation.key
kwargs['potid'] = potential.id
kwargs['potkey'] = potential.key

kwargs['elements'] = kwargs['symbols'] = potential.elements
#kwargs['elements'] = ['Fe', 'Fe']
#kwargs['symbols'] = ['Fe', 'nmFe']
kwargs['masses'] = [91.22, 1.008]

#kwargs['units'] =
#kwargs['atom_style'] = 

kwargs['pair_style'] = implementation.type.replace('LAMMPS pair_style ', '')
#kwargs['pair_style_terms'] = ['spline', 10000]

kwargs['comments'] = '\n'.join([f'Potential {implementation.id} listed in the NIST Interatomic Potentials Repository:',
                                f'https://www.ctcms.nist.gov/potentials/entry/{potential.id}/{implementation.id}.html',
                                f''])

#kwargs['dois'] = []
#for cit in potential.citations:
#    kwargs['dois'].append(cit.bib['doi'])

kwargs['artifacts'] = implementation.artifacts

Option #1 For pair potentials without parameter files, e.g lj/cut

- __interactions__ (*list*) Lists parameters associated with each unique interaction.  Each value is a dict with
    - __symbols__ (*list*) The two elemental model symbols to associate with the interaction.
    - __terms__ (*list*) The terms that appear on the pair_coeff command line for that interaction.

In [34]:
kwargs['interactions'] = [
    {'symbols': ['Co', 'Co'], 'terms': ['mie_Co-Co.table', 'ENTRY']},
    {'symbols': ['Cr', 'Cr'], 'terms': ['mie_Cr-Cr.table', 'ENTRY']},
    {'symbols': ['Fe', 'Fe'], 'terms': ['mie_Fe-Fe.table', 'ENTRY']},
    {'symbols': ['Mn', 'Mn'], 'terms': ['mie_Mn-Mn.table', 'ENTRY']},
    {'symbols': ['Ni', 'Ni'], 'terms': ['mie_Ni-Ni.table', 'ENTRY']},
    {'symbols': ['Co', 'Cr'], 'terms': ['mie_Co-Cr.table', 'ENTRY']},
    {'symbols': ['Co', 'Fe'], 'terms': ['mie_Co-Fe.table', 'ENTRY']},
    {'symbols': ['Co', 'Mn'], 'terms': ['mie_Co-Mn.table', 'ENTRY']},
    {'symbols': ['Co', 'Ni'], 'terms': ['mie_Co-Ni.table', 'ENTRY']},
    {'symbols': ['Cr', 'Fe'], 'terms': ['mie_Cr-Fe.table', 'ENTRY']},
    {'symbols': ['Cr', 'Mn'], 'terms': ['mie_Cr-Mn.table', 'ENTRY']},
    {'symbols': ['Cr', 'Ni'], 'terms': ['mie_Cr-Ni.table', 'ENTRY']},
    {'symbols': ['Fe', 'Mn'], 'terms': ['mie_Fe-Mn.table', 'ENTRY']},
    {'symbols': ['Fe', 'Ni'], 'terms': ['mie_Fe-Ni.table', 'ENTRY']},
    {'symbols': ['Mn', 'Ni'], 'terms': ['mie_Mn-Ni.table', 'ENTRY']},
]

Option #2 For potentials with single parameter files, e.g. eam/alloy

- __paramfile__ (*str*) The name (no path) of the potential's parameter file.

In [60]:
for file in filenames:
    print(file.name)

Pb_wang.ream
pair_ream.cpp
pair_ream.h


In [65]:
kwargs['paramfile'] = 'Pb_wang.ream'

Option #3 For potentials with a parameter file and a library file, e.g. meam

- __libfile__ (*str*) The name (no path) of the potential's library parameter file.
- __paramfile__ (*str, optional*) The name (no path) of the potential's parameter file, if one is used.

In [33]:
kwargs['libfile'] = 'library.meam'
kwargs['paramfile'] = 'ZrH.meam'

Option #4 For classic EAM potentials, i.e. eam

- __paramfiles__ (*list*) The names (no paths) of the parameter files for each unique element. Order should match order of elements.

In [ ]:
kwargs['paramfiles'] = []

Option #5 For OpenKIM potentials

- __kimid__ (*str*) The KIM model ID.

In [ ]:
kwargs['kimid'] = 

#### Build potential_LAMMPS

In [34]:
lammps_potential = potentials.build_lammps_potential(**kwargs).potential()

In [66]:
lammps_potential = potentials.buildrecord.potential_LAMMPS.ParamFileBuilder(**kwargs).potential()

### 5.2 Load potential_LAMMPS from database 

In [29]:
lammps_potential = db.get_lammps_potential(id='2023--Jana-R--Fe--LAMMPS--ipr1', verbose=True)

Found 1 matching potential_LAMMPS records in local library
Found 0 matching potential_LAMMPS_KIM records in local library
Built 0 lammps potentials for KIM models
Matching record retrieved from local


F:\Python-packages\yabadaba\yabadaba\database\LocalDatabase.py:138: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cache = cache.append(newrecords, sort=False).sort_values('name')


### 5.3 Load potential_LAMMPS from local file not in database 

In [44]:
fname = Path('2020--Groger-R--Co-Cr-Fe-Mn-Ni--LAMMPS--ipr1.json')

lammps_potential = potentials.record.PotentialLAMMPS(fname)

### 5.4 Review and save

In [34]:
lammps_potential.url = f'https://potentials.nist.gov/pid/rest/local/potentials/{lammps_potential.name}'

AttributeError: can't set attribute

#### Review model

In [35]:
print(lammps_potential.model.json(indent=2))

{
  "potential-LAMMPS": {
    "key": "7a2e3aa3-39dc-4bb9-ac9a-33a34d77396d",
    "id": "2014--Lee-B-M--Zr-H--LAMMPS--ipr2",
    "potential": {
      "key": "ebea64e3-886d-4005-9966-0d7e2bad13cc",
      "id": "2014--Lee-B-M-Lee-B-J--Zr-H"
    },
    "comments": "Potential 2014--Lee-B-M--Zr-H--LAMMPS--ipr2 listed in the NIST Interatomic Potentials Repository:\nhttps://www.ctcms.nist.gov/potentials/entry/2014--Lee-B-M-Lee-B-J--Zr-H/2014--Lee-B-M--Zr-H--LAMMPS--ipr2.html\n",
    "units": "metal",
    "atom_style": "atomic",
    "atom": [
      {
        "symbol": "Zr",
        "element": "Zr",
        "mass": 91.22
      },
      {
        "symbol": "H",
        "element": "H",
        "mass": 1.008
      }
    ],
    "pair_style": {
      "type": "meam"
    },
    "pair_coeff": {
      "term": [
        {
          "file": "library.meam"
        },
        {
          "option": "Zr H"
        },
        {
          "file": "ZrH.meam"
        },
        {
          "symbols": true
        

#### Save

In [31]:
db.save_lammps_potential(lammps_potential, filenames=filenames,  verbose=True)
db.upload_lammps_potential(lammps_potential, workspace=workspace, overwrite=True, verbose=True)

potential_LAMMPS record named 2023--Jana-R--Fe--LAMMPS--ipr1 added to https://potentials.nist.gov/
potential_LAMMPS record named 2023--Jana-R--Fe--LAMMPS--ipr1 assigned to workspace Global Public Workspace


In [ ]:
#db.delete_lammps_potential(lammps_potential, local=True, remote=True, verbose=True)

In [ ]:
db.download_lammps_potentials(getfiles=True, overwrite=True, verbose=True)

- - -

## 5. Actions

Actions are used by the website to note changes in content and representation. 

### 5.1 Build action

#### Option #1: new posting(s)

In [11]:
complete_potentials = [potential]

In [12]:
for complete_potential in complete_potentials:
    print(complete_potential.id)

2023--Nitol-M-S-Dang-K-Fensin-S-J-et-al--Sn


In [13]:
action = potentials.load_record('Action',
    type = 'new posting',
    potentials = complete_potentials,
    #date = datetime.date(2021, 10, 26),
    comment = "New posting for Sn",
)

#### Option #2: updated posting(s)

In [39]:
action = potentials.load_record('Action',
    type = 'updated posting',
    potentials = complete_potentials,
    date = None,
    comment = 'New version added that fixes the atomic mass of H',
)

#### Option #3: retraction

In [ ]:
action = potentials.load_record('Action',
    type = 'retraction',
    potentials = complete_potentials,
    date = None,        
    comment =,
)

#### Option #4: site change

In [ ]:
action = potentials.load_record('Action',
    type = 'site change',
    date = None,
    comment = 'IDs added for all non-LAMMPS implementations. The handling of fictional potentials made consistent with "real" potentials.',
    potentials = []
)

### 5.2 Look and save

In [14]:
action.build_model()
action.html(render=True)

In [15]:
db.upload_action(action, workspace=workspace, overwrite=True, verbose=True)

Action record named 2023-07-19 New posting for Sn added to https://potentials.nist.gov/
Action record named 2023-07-19 New posting for Sn assigned to workspace Global Public Workspace


- - -

## 6. Requests

Requests are used by the website to list when people are looking for specific models that we don't have.

### 6.1 Build request

In [42]:
request = potentials.load_record(
    'Request',
    #comment = 'for modeling large scale metal-metal diffusion and ballistic radiation damage',
    #date = datetime.date(2022, 11, 1),
    systems = [
        {
            #'formula': 'Y2Zr2O7',
            'elements': ['Cu', 'Ni', 'Zr'],
        },
        #{
        #    'formula': 'KNbO3',
        #    'elements': ['Cu', 'Zr', 'Ti', 'Be', 'Ni'],
        #},
    ]
)


### 6.2 Look and save

In [43]:
request.build_model()
display(HTML(request.html()))

In [44]:
db.upload_request(request, workspace=workspace, overwrite=True, verbose=True)

Request record named 2023-05-10 Cu Ni Zr added to https://potentials.nist.gov/
Request record named 2023-05-10 Cu Ni Zr assigned to workspace Global Public Workspace


## 7. Related models

In [34]:
import json

def add_related_models(localpath, interaction, model1, model2):
    
    with open(Path(localpath, 'site/related-interactions.json')) as f:
        related_models = json.load(f)
    
    # If no shared models for that interaction yet
    if interaction not in related_models:
        print('new interaction')
        related_models[interaction] = [[model1, model2]]
            
    else:
        int_models = related_models[interaction]
        
        # Search for set with one of the elements
        match = False
        for i in range(len(int_models)):
            modelset = int_models[i]
            if model1 in modelset:
                if match is False:
                    match = i
                else:
                    raise ValueError('given models found in multiple sets!')
            if model2 in modelset:
                if match is False or match == i:
                    match = i
                else:
                    raise ValueError('given models found in multiple sets!')
                    
        if match is False:
            print('new set')
            int_models.append(sorted([model1, model2]))
        else:
            if model1 not in int_models[match]:
                int_models[match] = sorted(int_models[match] + [model1])
                print('model1 added to existing set')
            elif model2 not in int_models[match]:
                int_models[match] = sorted(int_models[match] + [model2])
                print('model2 added to existing set')
            else:
                print('both models already in existing set')
    
    with open(Path(localpath, 'site/related-interactions.json'), 'w') as f:
        json.dump(related_models, fp=f)

In [40]:
interaction = 'Fe-Ni'
model1 = '2023--Tramontina-D-R-Deluigi-O-R-Pinzon-R-et-al--Fe-Cu-Ni'
model2 = '2009--Bonny-G-Pasianot-R-C-Castin-N-Malerba-L--Fe-Cu-Ni'
machine = 'desktop'

if machine == 'desktop':
    localpath = Path('F:/website/IPR-website/potentials/')
elif machine == 'laptop':
    localpath = Path('C:/Users/lmh1/Documents/website/IPR-website/potentials/')

add_related_models(localpath, interaction, model1, model2)

model1 added to existing set
